# Baselineonly

In [9]:
import matplotlib.pyplot as plt
import numpy as np
import surprise
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise import Dataset, Reader
import pandas as pd


reader = Reader(line_format='item user rating timestamp', sep=',', rating_scale=(0.5, 5), skip_lines=1)
#reader로 모델에 사용할 데이터 형식을 맞춰주다

data_folds = Dataset.load_from_file('C:/Users/kimjunseok/Downloads/mealion/ratings.csv',reader=reader)

trainset = data_folds.build_full_trainset()

bsl_options = {'method': 'als',
               'n_epochs': 2,
               'reg_u': 5,
               'reg_i': 2
               }

algo = surprise.BaselineOnly(bsl_options=bsl_options)


algo.fit(trainset)

ratings = pd.read_csv('C:/Users/kimjunseok/Downloads/mealion/ratings.csv')
restaurants = pd.read_csv('C:/Users/kimjunseok/Downloads/mealion/restaurants.csv')

#restaurantIds = ratings[ratings['userId']==9]['restaurantId']
#if restaurantIds[restaurantIds==42].count() == 0:
#    print('user id=9인 사람은 restaurant id=42에 대한 평점이 없음')
    
#print(restaurants[restaurants['restaurantId']==42])

def get_unvisited_surprise(score, restaurantId, userId):
    visited_restaurants = ratings[ratings['userId']==userId]['restaurantId'].tolist()
    total_restaurants =  restaurants['id'].tolist()
    unvisited_restaurants = [restaurant for restaurant in total_restaurants if restaurant not in visited_restaurants]
    print(f'특정 {userId}번 유저가 가본 식당 수: {len(visited_restaurants)}\n추천한 식당 개수: {len(unvisited_restaurants)}\n전체 식당수: {len(total_restaurants)}')
    
    return unvisited_restaurants

def recomm_restaurant_by_surprise(algo, userId,unvisited_restaurants, top_n=10 ):
    predictions = [algo.predict(str(userId), str(restaurantId)) for restaurantId in unvisited_restaurants]
    
    def sortkey_est(pred):
        return pred.est
    predictions.sort(key=sortkey_est, reverse=True)  #내림차순 , key 통해 평점을 기준으로 정렬
    top_predictions = predictions[:top_n]
    top_restaurant_ids = [int(pred.iid) for pred in top_predictions] #pred.iid = 아이템 아이디 , 식당 아이디
    top_restaurant_ratings = [pred.est for pred in top_predictions]  #pred.est = 예측평점
    top_restaurant_titles = restaurants[restaurants.id.isin(top_restaurant_ids)]['name']
    
    top_restaurant_preds = [(ids, np.round(rating,2) , name) for ids, rating, name in zip(top_restaurant_ids, top_restaurant_ratings, top_restaurant_titles)]  #zip: 두 그룹의 데이터를 서로 엮어주는 파이썬의 내장함수
    
    return top_restaurant_preds

unvisited_lst = get_unvisited_surprise(ratings,restaurants, 9) #get_unvisited_surprise의 score, restaurantId는 ratings 데이터 프레임에서 찾다
top_restaurant_preds = recomm_restaurant_by_surprise(algo, 9, unvisited_lst,
                                           top_n=10)

print('baseline-als')
print('#'*8,'Top-10 추천식당 리스트','#'*8)
for top_restaurant in top_restaurant_preds:
    print('* 추천 식당 ID: ', top_restaurant[0])
    print('* 추천 식당 이름:',top_restaurant[2])
    print('* 해당 식당의 예측평점: ', top_restaurant[1])
    print()

특정 9번 유저가 가본 식당 수: 243
추천한 식당 개수: 849616
전체 식당수: 849859
baseline-als
######## Top-10 추천식당 리스트 ########
* 추천 식당 ID:  348296
* 추천 식당 이름: 롱리브
* 해당 식당의 예측평점:  4.09

* 추천 식당 ID:  641985
* 추천 식당 이름: 올블루파스타
* 해당 식당의 예측평점:  4.01

* 추천 식당 ID:  618104
* 추천 식당 이름: 커피제이비
* 해당 식당의 예측평점:  4.0

* 추천 식당 ID:  307401
* 추천 식당 이름: 아리랑떡볶이 강대점
* 해당 식당의 예측평점:  4.0

* 추천 식당 ID:  506268
* 추천 식당 이름: 그린때때
* 해당 식당의 예측평점:  3.99

* 추천 식당 ID:  851033
* 추천 식당 이름: 레인보우피시
* 해당 식당의 예측평점:  3.96

* 추천 식당 ID:  557107
* 추천 식당 이름: 아웃뺑
* 해당 식당의 예측평점:  3.96

* 추천 식당 ID:  850505
* 추천 식당 이름: 꼴두바우 애막골점
* 해당 식당의 예측평점:  3.95

* 추천 식당 ID:  229606
* 추천 식당 이름: 벌집삼겹살
* 해당 식당의 예측평점:  3.94

* 추천 식당 ID:  850572
* 추천 식당 이름: 마릴본
* 해당 식당의 예측평점:  3.91



In [7]:
import matplotlib.pyplot as plt
import numpy as np
import surprise
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise import Dataset, Reader
import pandas as pd


reader = Reader(line_format='item user rating timestamp', sep=',', rating_scale=(0.5, 5), skip_lines=1)

data_folds = Dataset.load_from_file('C:/Users/kimjunseok/Downloads/mealion/ratings.csv',reader=reader)

trainset = data_folds.build_full_trainset()

bsl_options = {'method': 'als',
               'n_epochs': 2,
               'reg_u': 5,
               'reg_i': 2
               }

algo = surprise.BaselineOnly(bsl_options=bsl_options)


algo.fit(trainset)

ratings = pd.read_csv('C:/Users/kimjunseok/Downloads/mealion/ratings.csv')
restaurants = pd.read_csv('C:/Users/kimjunseok/Downloads/mealion/restaurants.csv')


def get_unvisited_surprise(score, restaurantId, userId):
    visited_restaurants = ratings[ratings['userId']==userId]['restaurantId'].tolist()
    total_restaurants =  restaurants['id'].tolist()
    unvisited_restaurants = [restaurant for restaurant in total_restaurants if restaurant not in visited_restaurants]
    return unvisited_restaurants

def recomm_restaurant_by_surprise(algo, userId,unvisited_restaurants, top_n=10 ):
    predictions = [algo.predict(str(userId), str(restaurantId)) for restaurantId in unvisited_restaurants]
    
    def sortkey_est(pred):
        return pred.est
    predictions.sort(key=sortkey_est, reverse=True)  
    top_predictions = predictions[:top_n]
    top_restaurant_ids = [int(pred.iid) for pred in top_predictions]
    top_restaurant_ratings = [pred.est for pred in top_predictions]  
    top_restaurant_titles = restaurants[restaurants.id.isin(top_restaurant_ids)]['name']
    
    top_restaurant_preds = [(ids, np.round(rating,2) , name) for ids, rating, name in zip(top_restaurant_ids, top_restaurant_ratings, top_restaurant_titles)]  
    
    return top_restaurant_preds

unvisited_lst = get_unvisited_surprise(ratings,restaurants, 9) 
top_restaurant_preds = recomm_restaurant_by_surprise(algo, 9, unvisited_lst,
                                           top_n=10)

top_restaurant_preds


Estimating biases using als...


[(19222, 4.07, '광주김밥'),
 (538214, 4.01, '미스터빈'),
 (850504, 3.94, '올블루파스타'),
 (307401, 3.93, '일편단심'),
 (506268, 3.9, '아리랑떡볶이 강대점'),
 (413378, 3.89, '명품탕수육'),
 (850572, 3.88, '네오피자퀵'),
 (850542, 3.87, '수아마노'),
 (545754, 3.86, '큰골집'),
 (35759, 3.84, '벌집삼겹살')]

In [3]:
top_restaurant_preds

[(19222, 4.0668455928889475, '광주김밥'),
 (538214, 4.011375857893731, '미스터빈'),
 (850504, 3.94261836508273, '올블루파스타'),
 (307401, 3.93388085738252, '일편단심'),
 (506268, 3.9036151608112584, '아리랑떡볶이 강대점'),
 (413378, 3.8897485112454726, '명품탕수육'),
 (850572, 3.884869843023049, '네오피자퀵'),
 (850542, 3.866888851525185, '수아마노'),
 (545754, 3.8566212100687554, '큰골집'),
 (35759, 3.843060395057522, '벌집삼겹살')]

# SVD 

### 충효님 추천 식당 (춘천 거주, 이용 횟수- 중간)

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import surprise
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise import Dataset, Reader
import pandas as pd

a=10000
reader = Reader(line_format='item user rating timestamp', sep=',', rating_scale=(0.5, 5), skip_lines=1)
#reader로 모델에 사용할 데이터 형식을 맞춰주다

data_folds = Dataset.load_from_file('C:/Users/kimjunseok/Downloads/mealion/ratings.csv',reader=reader)

trainset = data_folds.build_full_trainset()

sim_options = {'name': 'msd'}
algo = surprise.SVD(n_factors=20, n_epochs=40)




algo.fit(trainset)

ratings = pd.read_csv('C:/Users/kimjunseok/Downloads/mealion/ratings.csv')
restaurants = pd.read_csv('C:/Users/kimjunseok/Downloads/mealion2/restaurants.csv')

#restaurantIds = ratings[ratings['userId']==9]['restaurantId']
#if restaurantIds[restaurantIds==42].count() == 0:
#    print('user id=9인 사람은 restaurant id=42에 대한 평점이 없음')
    
#print(restaurants[restaurants['restaurantId']==42])

def get_unvisited_surprise(score, restaurantId, userId):
    visited_restaurants = ratings[ratings['userId']==userId]['restaurantId'].tolist()
    total_restaurants =  restaurants['id'].tolist()
    unvisited_restaurants = [restaurant for restaurant in total_restaurants if restaurant not in visited_restaurants]
    print(f'특정 {userId}번 유저가 가본 식당 수: {len(visited_restaurants)}\n추천한 식당 개수: {len(unvisited_restaurants)}\n전체 식당수: {len(total_restaurants)}')
    
    return unvisited_restaurants

def recomm_restaurant_by_surprise(algo, userId,unvisited_restaurants, top_n=10 ):
    predictions = [algo.predict(str(userId), str(restaurantId)) for restaurantId in unvisited_restaurants]
    
    def sortkey_est(pred):
        return pred.est
    predictions.sort(key=sortkey_est, reverse=True)  #내림차순 , key 통해 평점을 기준으로 정렬
    top_predictions = predictions[:top_n]
    top_restaurant_ids = [int(pred.iid) for pred in top_predictions] #pred.iid = 아이템 아이디 , 식당 아이디
    top_restaurant_ratings = [pred.est for pred in top_predictions]  #pred.est = 예측평점
    top_restaurant_titles = restaurants[restaurants.id.isin(top_restaurant_ids)]['name']
    
    top_restaurant_preds = [(ids, np.round(rating,2) , name) for ids, rating, name in zip(top_restaurant_ids, top_restaurant_ratings, top_restaurant_titles)]  #zip: 두 그룹의 데이터를 서로 엮어주는 파이썬의 내장함수
    
    return top_restaurant_preds

unvisited_lst = get_unvisited_surprise(ratings,restaurants, a) #get_unvisited_surprise의 score, restaurantId는 ratings 데이터 프레임에서 찾다
top_restaurant_preds = recomm_restaurant_by_surprise(algo, a, unvisited_lst,
                                           top_n=10)

print('SVD')
print('#'*8,'Top-10 추천식당 리스트','#'*8)
for top_restaurant in top_restaurant_preds:
    print('* 추천 식당 ID: ', top_restaurant[0])
    print('* 추천 식당 이름:',top_restaurant[2])
    print('* 해당 식당의 예측평점: ', top_restaurant[1])
    print()

특정 10000번 유저가 가본 식당 수: 6
추천한 식당 개수: 849862
전체 식당수: 849868
SVD
######## Top-10 추천식당 리스트 ########
* 추천 식당 ID:  12289
* 추천 식당 이름: 수누리감자탕
* 해당 식당의 예측평점:  5

* 추천 식당 ID:  19222
* 추천 식당 이름: 광주김밥
* 해당 식당의 예측평점:  5

* 추천 식당 ID:  35692
* 추천 식당 이름: Vincero
* 해당 식당의 예측평점:  5

* 추천 식당 ID:  35759
* 추천 식당 이름: 미스터빈
* 해당 식당의 예측평점:  5

* 추천 식당 ID:  196621
* 추천 식당 이름: 푸주옥
* 해당 식당의 예측평점:  5

* 추천 식당 ID:  196693
* 추천 식당 이름: 보영이네해물칼국수
* 해당 식당의 예측평점:  5

* 추천 식당 ID:  234764
* 추천 식당 이름: 니뽕내뽕
* 해당 식당의 예측평점:  5

* 추천 식당 ID:  264885
* 추천 식당 이름: 토바돈
* 해당 식당의 예측평점:  5

* 추천 식당 ID:  307401
* 추천 식당 이름: 올블루파스타
* 해당 식당의 예측평점:  5

* 추천 식당 ID:  314225
* 추천 식당 이름: 일미리금계찜닭
* 해당 식당의 예측평점:  5



### 팀장님 추천 식당  (춘천 거주, 이용 횟수 많음)

In [11]:
import matplotlib.pyplot as plt
import numpy as np
import surprise
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise import Dataset, Reader
import pandas as pd

a=9
reader = Reader(line_format='item user rating timestamp', sep=',', rating_scale=(0.5, 5), skip_lines=1)
#reader로 모델에 사용할 데이터 형식을 맞춰주다

data_folds = Dataset.load_from_file('C:/Users/kimjunseok/OneDrive/바탕 화면/컨플/데이터/mealion_data/밀리언데이터/ratings0923.csv',reader=reader)

trainset = data_folds.build_full_trainset()

sim_options = {'name': 'msd'}
algo = surprise.SVD(n_factors=20, n_epochs=40)

#KNN BASELINE
#sim_options = {'name': 'msd'}
#algo = surprise.KNNBaseline(sim_options = sim_options)
#bsl_options = {
#    'method': 'als',
#    'n_epochs': 2,
#    'reg_u' : 5,
#    'reg_i': 2
#}
#algo = surprise.BaselineOnly(bsl_options=bsl_options)


algo.fit(trainset)

ratings = pd.read_csv('C:/Users/kimjunseok/OneDrive/바탕 화면/컨플/데이터/mealion_data/밀리언데이터/ratings0923.csv')
restaurants = pd.read_csv('C:/Users/kimjunseok/OneDrive/바탕 화면/컨플/데이터/mealion_data/밀리언데이터/restaurants.csv')

#restaurantIds = ratings[ratings['userId']==9]['restaurantId']
#if restaurantIds[restaurantIds==42].count() == 0:
#    print('user id=9인 사람은 restaurant id=42에 대한 평점이 없음')
    
#print(restaurants[restaurants['restaurantId']==42])

def get_unvisited_surprise(score, restaurantId, userId):
    visited_restaurants = ratings[ratings['userId']==userId]['restaurantId'].tolist()
    total_restaurants =  restaurants['id'].tolist()
    unvisited_restaurants = [restaurant for restaurant in total_restaurants if restaurant not in visited_restaurants]
    print(f'특정 {userId}번 유저가 가본 식당 수: {len(visited_restaurants)}\n추천한 식당 개수: {len(unvisited_restaurants)}\n전체 식당수: {len(total_restaurants)}')
    
    return unvisited_restaurants

def recomm_restaurant_by_surprise(algo, userId,unvisited_restaurants, top_n=10 ):
    predictions = [algo.predict(str(userId), str(restaurantId)) for restaurantId in unvisited_restaurants]
    
    def sortkey_est(pred):
        return pred.est
    predictions.sort(key=sortkey_est, reverse=True)  #내림차순 , key 통해 평점을 기준으로 정렬
    top_predictions = predictions[:top_n]
    top_restaurant_ids = [int(pred.iid) for pred in top_predictions] #pred.iid = 아이템 아이디 , 식당 아이디
    top_restaurant_ratings = [pred.est for pred in top_predictions]  #pred.est = 예측평점
    top_restaurant_titles = restaurants[restaurants.id.isin(top_restaurant_ids)]['name']
    
    top_restaurant_preds = [(ids, np.round(rating,2) , name) for ids, rating, name in zip(top_restaurant_ids, top_restaurant_ratings, top_restaurant_titles)]  #zip: 두 그룹의 데이터를 서로 엮어주는 파이썬의 내장함수
    
    return top_restaurant_preds

unvisited_lst = get_unvisited_surprise(ratings,restaurants, a) #get_unvisited_surprise의 score, restaurantId는 ratings 데이터 프레임에서 찾다
top_restaurant_preds = recomm_restaurant_by_surprise(algo, a, unvisited_lst,
                                           top_n=10000)

print('SVD')
print('#'*8,'Top-10 추천식당 리스트','#'*8)
for top_restaurant in top_restaurant_preds:
    print('* 추천 식당 ID: ', top_restaurant[0])
    print('* 추천 식당 이름:',top_restaurant[2])
    print('* 해당 식당의 예측평점: ', top_restaurant[1])
    print()

Estimating biases using als...
특정 9번 유저가 가본 식당 수: 252
추천한 식당 개수: 849608
전체 식당수: 849859
SVD
######## Top-10 추천식당 리스트 ########
* 추천 식당 ID:  19222
* 추천 식당 이름: 프로포즈호프
* 해당 식당의 예측평점:  4.81

* 추천 식당 ID:  538214
* 추천 식당 이름: 착한짜장
* 해당 식당의 예측평점:  4.75

* 추천 식당 ID:  850504
* 추천 식당 이름: 해마로숯불바베큐
* 해당 식당의 예측평점:  4.69

* 추천 식당 ID:  413378
* 추천 식당 이름: 청암회관
* 해당 식당의 예측평점:  4.64

* 추천 식당 ID:  850572
* 추천 식당 이름: 배제현의웰빙낙지한마리수제비 강남신사본점
* 해당 식당의 예측평점:  4.62

* 추천 식당 ID:  850542
* 추천 식당 이름: 동화장
* 해당 식당의 예측평점:  4.61

* 추천 식당 ID:  545754
* 추천 식당 이름: 우림영양탕
* 해당 식당의 예측평점:  4.6

* 추천 식당 ID:  850761
* 추천 식당 이름: 아마노
* 해당 식당의 예측평점:  4.59

* 추천 식당 ID:  35759
* 추천 식당 이름: 피자마루 약수점
* 해당 식당의 예측평점:  4.59

* 추천 식당 ID:  538439
* 추천 식당 이름: 유래성
* 해당 식당의 예측평점:  4.58

* 추천 식당 ID:  545558
* 추천 식당 이름: 우리식당
* 해당 식당의 예측평점:  4.58

* 추천 식당 ID:  621099
* 추천 식당 이름: 크리스피크림 롯데백화점 청량리점
* 해당 식당의 예측평점:  4.57

* 추천 식당 ID:  553888
* 추천 식당 이름: 커피빈 교대점
* 해당 식당의 예측평점:  4.57

* 추천 식당 ID:  506268
* 추천 식당 이름: 대박곱창구이
* 해당 식당의 예측평점:  4.55

* 추천 식당 I

### 경기도, 서울 거주, 밀리언 이용- 많음 (맛본좌)

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import surprise
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise import Dataset, Reader
import pandas as pd

a=66
reader = Reader(line_format='item user rating timestamp', sep=',', rating_scale=(0.5, 5), skip_lines=1)
#reader로 모델에 사용할 데이터 형식을 맞춰주다

data_folds = Dataset.load_from_file('C:/Users/kimjunseok/Downloads/mealion/ratings.csv',reader=reader)

trainset = data_folds.build_full_trainset()

sim_options = {'name': 'msd'}
algo = surprise.SVD(n_factors=20, n_epochs=40)




algo.fit(trainset)

ratings = pd.read_csv('C:/Users/kimjunseok/Downloads/mealion/ratings.csv')
restaurants = pd.read_csv('C:/Users/kimjunseok/Downloads/mealion/restaurants.csv')

#restaurantIds = ratings[ratings['userId']==9]['restaurantId']
#if restaurantIds[restaurantIds==42].count() == 0:
#    print('user id=9인 사람은 restaurant id=42에 대한 평점이 없음')
    
#print(restaurants[restaurants['restaurantId']==42])

def get_unvisited_surprise(score, restaurantId, userId):
    visited_restaurants = ratings[ratings['userId']==userId]['restaurantId'].tolist()
    total_restaurants =  restaurants['id'].tolist()
    unvisited_restaurants = [restaurant for restaurant in total_restaurants if restaurant not in visited_restaurants]
    print(f'특정 {userId}번 유저가 가본 식당 수: {len(visited_restaurants)}\n추천한 식당 개수: {len(unvisited_restaurants)}\n전체 식당수: {len(total_restaurants)}')
    
    return unvisited_restaurants

def recomm_restaurant_by_surprise(algo, userId,unvisited_restaurants, top_n=10 ):
    predictions = [algo.predict(str(userId), str(restaurantId)) for restaurantId in unvisited_restaurants]
    
    def sortkey_est(pred):
        return pred.est
    predictions.sort(key=sortkey_est, reverse=True)  #내림차순 , key 통해 평점을 기준으로 정렬
    top_predictions = predictions[:top_n]
    top_restaurant_ids = [int(pred.iid) for pred in top_predictions] #pred.iid = 아이템 아이디 , 식당 아이디
    top_restaurant_ratings = [pred.est for pred in top_predictions]  #pred.est = 예측평점
    top_restaurant_titles = restaurants[restaurants.id.isin(top_restaurant_ids)]['name']
    
    top_restaurant_preds = [(ids, np.round(rating,2) , name) for ids, rating, name in zip(top_restaurant_ids, top_restaurant_ratings, top_restaurant_titles)]  #zip: 두 그룹의 데이터를 서로 엮어주는 파이썬의 내장함수
    
    return top_restaurant_preds

unvisited_lst = get_unvisited_surprise(ratings,restaurants, a) #get_unvisited_surprise의 score, restaurantId는 ratings 데이터 프레임에서 찾다
top_restaurant_preds = recomm_restaurant_by_surprise(algo, a, unvisited_lst,
                                           top_n=10)

print('SVD')
print('#'*8,'Top-10 추천식당 리스트','#'*8)
for top_restaurant in top_restaurant_preds:
    print('* 추천 식당 ID: ', top_restaurant[0])
    print('* 추천 식당 이름:',top_restaurant[2])
    print('* 해당 식당의 예측평점: ', top_restaurant[1])
    print()

특정 66번 유저가 가본 식당 수: 154
추천한 식당 개수: 849705
전체 식당수: 849859
SVD
######## Top-10 추천식당 리스트 ########
* 추천 식당 ID:  19222
* 추천 식당 이름: 광주김밥
* 해당 식당의 예측평점:  4.74

* 추천 식당 ID:  850811
* 추천 식당 이름: 회영루
* 해당 식당의 예측평점:  4.63

* 추천 식당 ID:  391811
* 추천 식당 이름: 브릭스피자
* 해당 식당의 예측평점:  4.62

* 추천 식당 ID:  506268
* 추천 식당 이름: 고봉민김밥인후평점
* 해당 식당의 예측평점:  4.62

* 추천 식당 ID:  204753
* 추천 식당 이름: 엄마의손길
* 해당 식당의 예측평점:  4.61

* 추천 식당 ID:  354542
* 추천 식당 이름: 아리랑떡볶이 강대점
* 해당 식당의 예측평점:  4.59

* 추천 식당 ID:  850542
* 추천 식당 이름: 명품탕수육
* 해당 식당의 예측평점:  4.57

* 추천 식당 ID:  553780
* 추천 식당 이름: 카페엘리
* 해당 식당의 예측평점:  4.54

* 추천 식당 ID:  385634
* 추천 식당 이름: 큰골집
* 해당 식당의 예측평점:  4.54

* 추천 식당 ID:  538214
* 추천 식당 이름: 팍스하우스
* 해당 식당의 예측평점:  4.53



### 신규 회원 - 이용수 없음

In [7]:
import matplotlib.pyplot as plt
import numpy as np
import surprise
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise import Dataset, Reader
import pandas as pd

a=835
reader = Reader(line_format='item user rating timestamp', sep=',', rating_scale=(0.5, 5), skip_lines=1)
#reader로 모델에 사용할 데이터 형식을 맞춰주다

data_folds = Dataset.load_from_file('C:/Users/kimjunseok/Downloads/mealion/ratings.csv',reader=reader)

trainset = data_folds.build_full_trainset()

sim_options = {'name': 'msd'}
algo = surprise.SVD(n_factors=20, n_epochs=40)




algo.fit(trainset)

ratings = pd.read_csv('C:/Users/kimjunseok/Downloads/mealion/ratings.csv')
restaurants = pd.read_csv('C:/Users/kimjunseok/Downloads/mealion/restaurants.csv')

#restaurantIds = ratings[ratings['userId']==9]['restaurantId']
#if restaurantIds[restaurantIds==42].count() == 0:
#    print('user id=9인 사람은 restaurant id=42에 대한 평점이 없음')
    
#print(restaurants[restaurants['restaurantId']==42])

def get_unvisited_surprise(score, restaurantId, userId):
    visited_restaurants = ratings[ratings['userId']==userId]['restaurantId'].tolist()
    total_restaurants =  restaurants['id'].tolist()
    unvisited_restaurants = [restaurant for restaurant in total_restaurants if restaurant not in visited_restaurants]
    print(f'특정 {userId}번 유저가 가본 식당 수: {len(visited_restaurants)}\n추천한 식당 개수: {len(unvisited_restaurants)}\n전체 식당수: {len(total_restaurants)}')
    
    return unvisited_restaurants

def recomm_restaurant_by_surprise(algo, userId,unvisited_restaurants, top_n=10 ):
    predictions = [algo.predict(str(userId), str(restaurantId)) for restaurantId in unvisited_restaurants]
    
    def sortkey_est(pred):
        return pred.est
    predictions.sort(key=sortkey_est, reverse=True)  #내림차순 , key 통해 평점을 기준으로 정렬
    top_predictions = predictions[:top_n]
    top_restaurant_ids = [int(pred.iid) for pred in top_predictions] #pred.iid = 아이템 아이디 , 식당 아이디
    top_restaurant_ratings = [pred.est for pred in top_predictions]  #pred.est = 예측평점
    top_restaurant_titles = restaurants[restaurants.id.isin(top_restaurant_ids)]['name']
    
    top_restaurant_preds = [(ids, np.round(rating,2) , name) for ids, rating, name in zip(top_restaurant_ids, top_restaurant_ratings, top_restaurant_titles)]  #zip: 두 그룹의 데이터를 서로 엮어주는 파이썬의 내장함수
    
    return top_restaurant_preds

unvisited_lst = get_unvisited_surprise(ratings,restaurants, a) #get_unvisited_surprise의 score, restaurantId는 ratings 데이터 프레임에서 찾다
top_restaurant_preds = recomm_restaurant_by_surprise(algo, a, unvisited_lst,
                                           top_n=10)

print('SVD')
print('#'*8,'Top-10 추천식당 리스트','#'*8)
for top_restaurant in top_restaurant_preds:
    print('* 추천 식당 ID: ', top_restaurant[0])
    print('* 추천 식당 이름:',top_restaurant[2])
    print('* 해당 식당의 예측평점: ', top_restaurant[1])
    print()

특정 835번 유저가 가본 식당 수: 0
추천한 식당 개수: 849859
전체 식당수: 849859
SVD
######## Top-10 추천식당 리스트 ########
* 추천 식당 ID:  506268
* 추천 식당 이름: 올블루파스타
* 해당 식당의 예측평점:  4.69

* 추천 식당 ID:  538214
* 추천 식당 이름: 신남큰집궁중삼계탕
* 해당 식당의 예측평점:  4.62

* 추천 식당 ID:  354542
* 추천 식당 이름: 브릭스피자
* 해당 식당의 예측평점:  4.62

* 추천 식당 ID:  307401
* 추천 식당 이름: 원조퇴골집오리숯불구이
* 해당 식당의 예측평점:  4.59

* 추천 식당 ID:  506176
* 추천 식당 이름: 어라운드키친
* 해당 식당의 예측평점:  4.59

* 추천 식당 ID:  466442
* 추천 식당 이름: 본래순대
* 해당 식당의 예측평점:  4.58

* 추천 식당 ID:  385828
* 추천 식당 이름: 아리랑떡볶이 강대점
* 해당 식당의 예측평점:  4.55

* 추천 식당 ID:  538383
* 추천 식당 이름: 명품탕수육
* 해당 식당의 예측평점:  4.55

* 추천 식당 ID:  321332
* 추천 식당 이름: 생곡막국수
* 해당 식당의 예측평점:  4.54

* 추천 식당 ID:  545897
* 추천 식당 이름: 육림객잔
* 해당 식당의 예측평점:  4.54



추천 알고리즘
- 1.사용자의 거주지를 고려하지 않는다 -> 서울, 경기도 거주민한데 춘천 식당 리스트가 뜬다.
- 사용자마다 추천 리스트는 다르다
- 2. 데이터 양이 적어 , 겹치는 추천 리스트 식당이름이 종종 있다.
-> 데이터(가본 식당, 평점) 가 더 많이 필요하다 : 다른 곳에서 데이터를 가져와 붙이다?
-> 정확도를 좀 더 높이다
: 앙상블 모형을 쓰다? : 


-> 거주지를 고려한 추천 리스트? 
(평점과 거리를 동시에 고려한 리스트? 평점과 거리에 가중치를 리스트를 만들어야 하나?)
(거리 데이터도 불러와서 수치데이터로 바꾼 다음에 계산 해주는 함수를 만들어야 하나..?)

# KNN-Baseline

In [14]:
import matplotlib.pyplot as plt
import numpy as np
import surprise
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise import Dataset, Reader
import pandas as pd

a=9
reader = Reader(line_format='item user rating timestamp', sep=',', rating_scale=(0.5, 5), skip_lines=1)
#reader로 모델에 사용할 데이터 형식을 맞춰주다

data_folds = Dataset.load_from_file('C:/Users/kimjunseok/Downloads/mealion/ratings.csv',reader=reader)

trainset = data_folds.build_full_trainset()

sim_options = {'name': 'msd'}
algo = surprise.KNNBaseline(sim_options = sim_options)




algo.fit(trainset)

ratings = pd.read_csv('C:/Users/kimjunseok/Downloads/mealion/ratings.csv')
restaurants = pd.read_csv('C:/Users/kimjunseok/Downloads/mealion/restaurants.csv')

#restaurantIds = ratings[ratings['userId']==9]['restaurantId']
#if restaurantIds[restaurantIds==42].count() == 0:
#    print('user id=9인 사람은 restaurant id=42에 대한 평점이 없음')
    
#print(restaurants[restaurants['restaurantId']==42])

def get_unvisited_surprise(score, restaurantId, userId):
    visited_restaurants = ratings[ratings['userId']==userId]['restaurantId'].tolist()
    total_restaurants =  restaurants['id'].tolist()
    unvisited_restaurants = [restaurant for restaurant in total_restaurants if restaurant not in visited_restaurants]
    print(f'특정 {userId}번 유저가 가본 식당 수: {len(visited_restaurants)}\n추천한 식당 개수: {len(unvisited_restaurants)}\n전체 식당수: {len(total_restaurants)}')
    
    return unvisited_restaurants

def recomm_restaurant_by_surprise(algo, userId,unvisited_restaurants, top_n=10 ):
    predictions = [algo.predict(str(userId), str(restaurantId)) for restaurantId in unvisited_restaurants]
    
    def sortkey_est(pred):
        return pred.est
    predictions.sort(key=sortkey_est, reverse=True)  #내림차순 , key 통해 평점을 기준으로 정렬
    top_predictions = predictions[:top_n]
    top_restaurant_ids = [int(pred.iid) for pred in top_predictions] #pred.iid = 아이템 아이디 , 식당 아이디
    top_restaurant_ratings = [pred.est for pred in top_predictions]  #pred.est = 예측평점
    top_restaurant_titles = restaurants[restaurants.id.isin(top_restaurant_ids)]['name']
    
    top_restaurant_preds = [(ids, np.round(rating,2) , name) for ids, rating, name in zip(top_restaurant_ids, top_restaurant_ratings, top_restaurant_titles)]  #zip: 두 그룹의 데이터를 서로 엮어주는 파이썬의 내장함수
    
    return top_restaurant_preds

unvisited_lst = get_unvisited_surprise(ratings,restaurants, a) #get_unvisited_surprise의 score, restaurantId는 ratings 데이터 프레임에서 찾다
top_restaurant_preds = recomm_restaurant_by_surprise(algo, a, unvisited_lst,
                                           top_n=10)

print('KNN-Baseline')
print('#'*8,'Top-10 추천식당 리스트','#'*8)
for top_restaurant in top_restaurant_preds:
    print('* 추천 식당 ID: ', top_restaurant[0])
    print('* 추천 식당 이름:',top_restaurant[2])
    print('* 해당 식당의 예측평점: ', top_restaurant[1])
    print()

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
특정 9번 유저가 가본 식당 수: 243
추천한 식당 개수: 849616
전체 식당수: 849859
KNN-Baseline
######## Top-10 추천식당 리스트 ########
* 추천 식당 ID:  348296
* 추천 식당 이름: 고드래미족발
* 해당 식당의 예측평점:  4.56

* 추천 식당 ID:  621099
* 추천 식당 이름: 커피제이비
* 해당 식당의 예측평점:  4.53

* 추천 식당 ID:  850611
* 추천 식당 이름: 투썸플레이스
* 해당 식당의 예측평점:  4.53

* 추천 식당 ID:  850572
* 추천 식당 이름: 맛자랑떡볶이
* 해당 식당의 예측평점:  4.48

* 추천 식당 ID:  538214
* 추천 식당 이름: 감
* 해당 식당의 예측평점:  4.47

* 추천 식당 ID:  189275
* 추천 식당 이름: 먹고먹고떡볶이
* 해당 식당의 예측평점:  4.47

* 추천 식당 ID:  391563
* 추천 식당 이름: 명품탕수육
* 해당 식당의 예측평점:  4.47

* 추천 식당 ID:  397002
* 추천 식당 이름: 인창동불곱창하우스
* 해당 식당의 예측평점:  4.47

* 추천 식당 ID:  525918
* 추천 식당 이름: 벌집삼겹살
* 해당 식당의 예측평점:  4.47

* 추천 식당 ID:  534443
* 추천 식당 이름: Connple
* 해당 식당의 예측평점:  4.47



# 벡엔드 보낸 알고리즘 

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import surprise
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise import Dataset, Reader
import pandas as pd

a=9
reader = Reader(line_format='item user rating timestamp', sep=',', rating_scale=(0.5, 5), skip_lines=1)

data_folds = Dataset.load_from_file('C:/Users/kimjunseok/OneDrive/바탕 화면/컨플/데이터/mealion_data/밀리언데이터/ratings.csv',reader=reader)

trainset = data_folds.build_full_trainset()

sim_options = {'name': 'msd'}
algo = surprise.SVD(n_factors=20, n_epochs=40)




algo.fit(trainset)

ratings = pd.read_csv('C:/Users/kimjunseok/OneDrive/바탕 화면/컨플/데이터/mealion_data/밀리언데이터/ratings.csv')
restaurants = pd.read_csv('C:/Users/kimjunseok/OneDrive/바탕 화면/컨플/데이터/mealion_data/밀리언데이터/restaurants.csv')



def get_unvisited_surprise(score, restaurantId, userId):
    visited_restaurants = ratings[ratings['userId']==userId]['restaurantId'].tolist()
    total_restaurants =  restaurants['id'].tolist()
    unvisited_restaurants = [restaurant for restaurant in total_restaurants if restaurant not in visited_restaurants]
    print(f'특정 {userId}번 유저가 가본 식당 수: {len(visited_restaurants)}\n추천한 식당 개수: {len(unvisited_restaurants)}\n전체 식당수: {len(total_restaurants)}')
    
    return unvisited_restaurants

def recomm_restaurant_by_surprise(algo, userId,unvisited_restaurants, top_n=10 ):
    predictions = [algo.predict(str(userId), str(restaurantId)) for restaurantId in unvisited_restaurants]
    
    def sortkey_est(pred):
        return pred.est
    predictions.sort(key=sortkey_est, reverse=True) 
    top_predictions = predictions[:top_n]
    top_restaurant_ids = [int(pred.iid) for pred in top_predictions]
    top_restaurant_ratings = [pred.est for pred in top_predictions]  
    top_restaurant_titles = restaurants[restaurants.id.isin(top_restaurant_ids)]['name']
    
    top_restaurant_preds = [(ids, np.round(rating,2) , name) for ids, rating, name in zip(top_restaurant_ids, top_restaurant_ratings, top_restaurant_titles)]  
    
    return top_restaurant_preds

unvisited_lst = get_unvisited_surprise(ratings,restaurants, a) 
top_restaurant_preds = recomm_restaurant_by_surprise(algo, a, unvisited_lst,
                                           top_n=10)

top_restaurant_preds


특정 9번 유저가 가본 식당 수: 243
추천한 식당 개수: 849616
전체 식당수: 849859


[(506268, 3.99, '올블루파스타'),
 (506419, 3.92, '대빵커돈까스'),
 (850611, 3.85, '아리랑떡볶이 강대점'),
 (307401, 3.85, '흥주선비국밥 남춘천점'),
 (850674, 3.81, '램스테이블'),
 (538214, 3.81, '명품탕수육'),
 (850505, 3.81, '꼴두바우 애막골점'),
 (530427, 3.79, '카페 딜라이트'),
 (439626, 3.78, 'Connple'),
 (850514, 3.75, '니뽕내뽕 선릉점')]

In [ ]:
#1.고객 history 기반으로 근처 맛집 추천
#사용자가 추천 했던 식당 기반으로 근처 지역 추천 고려

#2.1번 지역 추천, 2번 지역 추천 
# 미리 계산한 함수 형태
#지역별로 필터링 해서 1번 지역에 대한 추천 , 2번 지연에 대한 추천

In [6]:

import matplotlib.pyplot as plt
import numpy as np
import surprise
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise import Dataset, Reader
import pandas as pd

reader = Reader(line_format='item user rating timestamp', sep=',', rating_scale=(0.5, 5), skip_lines=1)

data_folds = Dataset.load_from_file('C:/Users/kimjunseok/Downloads/mealion/ratings.csv',reader=reader)

trainset = data_folds.build_full_trainset()

sim_options = {'name': 'msd'}
algo = surprise.SVD(n_factors=20, n_epochs=40)




algo.fit(trainset)

ratings = pd.read_csv('C:/Users/kimjunseok/Downloads/mealion/ratings.csv')
restaurants = pd.read_csv('C:/Users/kimjunseok/Downloads/mealion/restaurants.csv')



### 1번째 방법, 가본 식당이 아닌 전체 식당으로 추천 하다

In [33]:

a=1


All_restaurants =  restaurants['id'].tolist()

def recomm_restaurant_by_surprise(algo, userId,All_restaurants, top_n=10000 ):
    predictions = [algo.predict(str(userId), str(restaurantId)) for restaurantId in All_restaurants]
    
    def sortkey_est(pred):
        return pred.est
    predictions.sort(key=sortkey_est, reverse=True) 
    top_predictions = predictions[:top_n]
    top_restaurant_ids = [int(pred.iid) for pred in top_predictions]
    top_restaurant_ratings = [pred.est for pred in top_predictions]  
    top_restaurant_titles = restaurants[restaurants.id.isin(top_restaurant_ids)]['name']
    
    top_restaurant_preds = [(ids, np.round(rating,2) , name) for ids, rating, name in zip(top_restaurant_ids, top_restaurant_ratings, top_restaurant_titles)]  
    
    return top_restaurant_preds

unvisited_lst = get_unvisited_surprise(ratings,restaurants, a) 
top_restaurant_preds = recomm_restaurant_by_surprise(algo, a, unvisited_lst,
                                           top_n=10000)

top_restaurant_preds


특정 1번 유저가 가본 식당 수: 123
추천한 식당 개수: 849736
전체 식당수: 849859


[(506176, 4.28, '프로포즈호프'),
 (466442, 4.25, '착한짜장'),
 (321332, 4.25, '해마로숯불바베큐'),
 (851096, 4.2, '청암회관'),
 (19222, 4.18, '배제현의웰빙낙지한마리수제비 강남신사본점'),
 (328221, 4.17, '동화장'),
 (220221, 4.17, '우림영양탕'),
 (545754, 4.16, '아마노'),
 (850542, 4.16, '피자마루 약수점'),
 (502164, 4.13, '유래성'),
 (850514, 4.13, '우리식당'),
 (354856, 4.13, '크리스피크림 롯데백화점 청량리점'),
 (354954, 4.13, '커피빈 교대점'),
 (496288, 4.12, '대박곱창구이'),
 (850541, 4.12, '파스쿠찌 디큐브아트센터점'),
 (850653, 4.12, '롯데리아 신림역점'),
 (850522, 4.12, 'JBar'),
 (154636, 4.12, '소노야코엑스몰점'),
 (3664, 4.12, '모닝김밥'),
 (461608, 4.12, '완도세꼬시'),
 (850504, 4.11, '맛생맛사'),
 (456741, 4.1, '파파존스 고덕점'),
 (196621, 4.1, '롯데리아행당역점'),
 (557169, 4.1, '다래식당'),
 (154577, 4.1, '김가네우면네이처힐'),
 (494283, 4.1, '메머드커피'),
 (850760, 4.09, '사계절'),
 (35759, 4.09, '벌떼식당'),
 (321210, 4.09, '카페필로네'),
 (475003, 4.09, '요거프레소'),
 (234764, 4.09, '본도시락'),
 (373424, 4.08, '금조홍어'),
 (272339, 4.08, '구의식당'),
 (315357, 4.08, '은하노래광장주점'),
 (850578, 4.08, '서울과학기술대학교제2학생식당'),
 (506390, 4.08, '초밥&참치전문점연'),
 (413378, 4.0

In [35]:
len(top_restaurant_preds)

10000

In [18]:
total_restaurants =  restaurants['id'].tolist()
restaurants['id'].tolist()


[1,
 7,
 8,
 10,
 12,
 13,
 14,
 16,
 17,
 19,
 21,
 22,
 25,
 27,
 31,
 34,
 37,
 39,
 40,
 44,
 46,
 47,
 48,
 52,
 53,
 54,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 201,
 202,
 203,
 204,
 205,
 206,
 207,
 208,
 209,

In [19]:
unvisited_restaurants = [restaurant for restaurant in total_restaurants if restaurant not in visited_restaurants]
unvisited_restaurants

[1,
 7,
 8,
 10,
 12,
 13,
 14,
 16,
 17,
 19,
 21,
 22,
 25,
 27,
 31,
 34,
 37,
 39,
 40,
 44,
 46,
 47,
 48,
 52,
 53,
 54,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 201,
 202,
 203,
 204,
 205,
 206,
 207,
 208,
 209,

In [25]:
ratings

,restaurantId,userId,score,otherScores
0,506184,3.0,4.0,0|0|0|0
1,341523,3.0,3.0,3|0|3|2
2,385515,3.0,5.0,3|0|2|3
3,514251,1.0,5.0,3|0|3|2
4,341523,1.0,4.0,0|0|0|0
...,...,...,...,...
3627,850776,532.0,5.0,3|3|3|3
3628,851099,330.0,4.5,3|3|2|2
3629,698116,330.0,4.0,2|2|2|3
3630,668776,330.0,5.0,3|3|3|3


In [23]:
restaurants

,id,name,address
0,1,프로포즈호프,서울특별시 금천구 가산로 34-6
1,7,착한짜장,서울특별시 성동구 아차산로7길 17-1
2,8,해마로숯불바베큐,서울특별시 성동구 금호산길 62
3,10,청암회관,서울특별시 노원구 한글비석로8길 29
4,12,배제현의웰빙낙지한마리수제비 강남신사본점,서울특별시 강남구 강남대로152길 19
...,...,...,...
849854,851095,화월비,강원 춘천시 애막골길3번길 6-1
849855,851096,어흥식당,서울특별시 성북구 고려대로27길 40
849856,851097,하이브로우 버거하우스,강원도 강릉시 유죽길 455-12
849857,851098,소바식당,서울특별시 성동구 연무장7가길 6


In [ ]:
#평가한 식당 ID 리스트를 뽑다.
#
#유저마다 (사람들이 평가한) 식당들 예상별점 출력되도록 코딩 
#모든 유저- 평가한(리뷰, 별점)식당들에 대해 예상 별점을 출력하다 for range
#모든 유저- 유저별 식당 id- 유저별 식당 id의 예상별점을 데이터 프레임으로 만들다

In [66]:
users = pd.read_csv('C:/Users/kimjunseok/Downloads/mealion/users.csv')
users

,id,nickname
0,1,AMGN
1,3,곰두리
2,4,맛있으면우는고양이
3,5,밀리언
4,6,생갈치1호의행방불명
...,...,...
848,886,흰뱀
849,887,이찬규
850,888,yurim
851,889,비빅비비빅


## 모든 유저 top10000 저장, 예측 평점 알고리즘

In [102]:



All_restaurants =  restaurants['id'].tolist()


    
def recomm_restaurant_by_surprise(algo, userId,All_restaurants, top_n=10000 ):
    predictions = [algo.predict(str(userId), str(restaurantId)) for restaurantId in unvisited_restaurants]
    
    
    def sortkey_est(pred):
        return pred.est
    predictions.sort(key=sortkey_est, reverse=True) 
    top_predictions = predictions[:top_n]
    top_restaurant_ids = [int(pred.iid) for pred in top_predictions]
    top_restaurant_ratings = [pred.est for pred in top_predictions]  
    top_restaurant_titles = restaurants[restaurants.id.isin(top_restaurant_ids)]['name']
    user_id = i
    
    top_restaurant_preds = [( id_restaurant, np.round(rating,2) , name) for  id_restaurant, rating, name in zip(top_restaurant_ids, top_restaurant_ratings, top_restaurant_titles)]  
    
    return top_restaurant_preds

def make_dataframe(top_restaurant_preds, user_id):
    col_name = ['restaurant_id', 'rating','restaurant_name']
    list_df = pd.DataFrame(top_restaurant_preds, columns=col_name)
    list_df['user_id'] = user_id#user_number
    
    return list_df

def make_big_dataframe():
    for j in range(len(users['id'])):
        i = users['id'][j]
        predict =recomm_restaurant_by_surprise(algo, i ,All_restaurants , top_n=10000)
        list_df = make_dataframe(predict, i)
        if i ==1:
            concat_dataframe = list_df 
        else:
            
            concat_dataframe = pd.concat([concat_dataframe,list_df])
        print(concat_dataframe)
    return concat_dataframe
#top_restaurant_preds = recomm_restaurant_by_surprise(algo, user_number, All_restaurants,top_n=10000)

#top_restaurant_preds
#make_dataframe(top_restaurant_preds, i)

In [103]:
make_big_dataframe()

      restaurant_id  rating        restaurant_name  user_id
0            506176    4.28                 프로포즈호프        1
1            466442    4.25                   착한짜장        1
2            321332    4.25               해마로숯불바베큐        1
3            851096    4.20                   청암회관        1
4             19222    4.18  배제현의웰빙낙지한마리수제비 강남신사본점        1
...             ...     ...                    ...      ...
9995           9057    3.76            오프넌(offnen)        1
9996           9058    3.76             포식(구리인창본점)        1
9997           9059    3.76                 툭툭누들타이        1
9998           9060    3.76                   어흥식당        1
9999           9061    3.76            하이브로우 버거하우스        1

[10000 rows x 4 columns]
      restaurant_id  rating        restaurant_name  user_id
0            506176    4.28                 프로포즈호프        1
1            466442    4.25                   착한짜장        1
2            321332    4.25               해마로숯불바베큐        1
3            8

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import surprise
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise import Dataset, Reader
import pandas as pd

reader = Reader(line_format='item user rating timestamp', sep=',', rating_scale=(0.5, 5), skip_lines=1)

data_folds = Dataset.load_from_file('C:/Users/kimjunseok/Downloads/mealion/ratings.csv',reader=reader)

trainset = data_folds.build_full_trainset()

sim_options = {'name': 'msd'}
algo = surprise.SVD(n_factors=20, n_epochs=40)




algo.fit(trainset)

ratings = pd.read_csv('C:/Users/kimjunseok/Downloads/mealion/ratings.csv')
restaurants = pd.read_csv('C:/Users/kimjunseok/Downloads/mealion/restaurants.csv')



All_restaurants =  restaurants['id'].tolist()


    
def recomm_restaurant_by_surprise(algo, userId,All_restaurants, top_n=10000 ):
    predictions = [algo.predict(str(userId), str(restaurantId)) for restaurantId in unvisited_restaurants]
    
    
    def sortkey_est(pred):
        return pred.est
    predictions.sort(key=sortkey_est, reverse=True) 
    top_predictions = predictions[:top_n]
    top_restaurant_ids = [int(pred.iid) for pred in top_predictions]
    top_restaurant_ratings = [pred.est for pred in top_predictions]  
    top_restaurant_titles = restaurants[restaurants.id.isin(top_restaurant_ids)]['name']
    user_id = i
    
    top_restaurant_preds = [( id_restaurant, np.round(rating,2) , name) for  id_restaurant, rating, name in zip(top_restaurant_ids, top_restaurant_ratings, top_restaurant_titles)]  
    
    return top_restaurant_preds

def make_dataframe(top_restaurant_preds, user_id):
    col_name = ['restaurant_id', 'rating','restaurant_name']
    list_df = pd.DataFrame(top_restaurant_preds, columns=col_name)
    list_df['user_id'] = user_id#user_number
    
    return list_df

def make_big_dataframe():
    for j in range(len(users['id'])):
        i = users['id'][j]
        predict =recomm_restaurant_by_surprise(algo, i ,All_restaurants , top_n=10000)
        list_df = make_dataframe(predict, i)
        if i ==1:
            concat_dataframe = list_df 
        else:
            
            concat_dataframe = pd.concat([concat_dataframe,list_df])
        print(concat_dataframe)
    return concat_dataframe
#top_restaurant_preds = recomm_restaurant_by_surprise(algo, user_number, All_restaurants,top_n=10000)

#top_restaurant_preds
#make_dataframe(top_restaurant_preds, i)

In [54]:
user_number=1
col_name = ['restaurant_id', 'rating','restaurant_name']
#top_restaurant_preds
list_df = pd.DataFrame(top_restaurant_preds, columns=col_name)
list_df['user_id'] = user_number
list_df

,restaurant_id,rating,restaurant_name,user_id
0,506176,4.28,프로포즈호프,1
1,466442,4.25,착한짜장,1
2,321332,4.25,해마로숯불바베큐,1
3,851096,4.20,청암회관,1
4,19222,4.18,배제현의웰빙낙지한마리수제비 강남신사본점,1
...,...,...,...,...
9995,9057,3.76,오프넌(offnen),1
9996,9058,3.76,포식(구리인창본점),1
9997,9059,3.76,툭툭누들타이,1
9998,9060,3.76,어흥식당,1


In [85]:
for i in range(users['id'][i]):
    print(users['id'][i])

1
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
20
21
22
24
25
26
27
28
29
30
31
32
34
37
38
39
41
43
44
45
46
47
49
50
51
52
53
54
55
56
57
58
59
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
85
86
87
88
89
90
91
92
93
94
95
96
97
99
100
102
103
104
105
106
107
108
109
110
111
112
113
114
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
166
168
169
170
171
172
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
30

KeyError: 853

In [84]:
for i in range(len(users['id'])):
    users['id'][i]


In [83]:
users['id']

0        1
1        3
2        4
3        5
4        6
      ... 
848    886
849    887
850    888
851    889
852    890
Name: id, Length: 853, dtype: int64

In [99]:
concat_dataframe

NameError: name 'concat_dataframe' is not defined

In [34]:
import matplotlib.pyplot as plt
import numpy as np
import surprise
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise import Dataset, Reader
import pandas as pd

a=9
reader = Reader(line_format='item user rating timestamp', sep=',', rating_scale=(0.5, 5), skip_lines=1)
#reader로 모델에 사용할 데이터 형식을 맞춰주다

data_folds = Dataset.load_from_file('C:/Users/kimjunseok/OneDrive/바탕 화면/컨플/데이터/mealion_data/밀리언데이터/ratings0923.csv',reader=reader)

trainset = data_folds.build_full_trainset()

sim_options = {'name': 'msd'}
algo = surprise.SVD(n_factors=20, n_epochs=40)

#bsl_options = {
#    'method': 'als',
#    'n_epochs': 2,
#    'reg_u' : 5,
#    'reg_i': 2
#}
#algo = surprise.BaselineOnly(bsl_options=bsl_options)


#sim_options = {'name': 'msd'}
#algo = surprise.KNNBaseline(sim_options = sim_options)
algo.fit(trainset)


In [37]:
uid = str(9)
iid = str(521844)

pred = algo.predict(uid, iid)
print(pred)

user: 9          item: 521844     r_ui = None   est = 3.72   {'was_impossible': False}
